# Construir com Modelos Mistral

## Introdução

Nesta lição vais aprender:
- Explorar os diferentes Modelos Mistral
- Compreender os casos de uso e cenários para cada modelo
- Exemplos de código mostram as características únicas de cada modelo.


## Os Modelos Mistral

Nesta lição, vamos explorar 3 modelos Mistral diferentes: **Mistral Large**, **Mistral Small** e **Mistral Nemo**.

Cada um destes modelos está disponível gratuitamente no marketplace de Modelos do Github. O código neste notebook irá utilizar estes modelos para executar o código. Aqui tens mais detalhes sobre como usar os Modelos do Github para [prototipar com modelos de IA](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).


## Mistral Large 2 (2407)
O Mistral Large 2 é atualmente o modelo de topo da Mistral e foi concebido para utilização empresarial.

Este modelo é uma melhoria em relação ao Mistral Large original, oferecendo:
- Janela de contexto maior - 128k vs 32k
- Melhor desempenho em tarefas de Matemática e Programação - precisão média de 76,9% vs 60,4%
- Melhor desempenho multilingue - as línguas incluem: inglês, francês, alemão, espanhol, italiano, português, neerlandês, russo, chinês, japonês, coreano, árabe e hindi.

Com estas características, o Mistral Large destaca-se em:
- *Geração aumentada por recuperação (RAG)* - devido à janela de contexto maior
- *Chamadas de função* - este modelo tem chamadas de função nativas, permitindo integração com ferramentas externas e APIs. Estas chamadas podem ser feitas em paralelo ou de forma sequencial, uma após a outra.
- *Geração de código* - este modelo é excelente na geração de código em Python, Java, TypeScript e C++.


Neste exemplo, estamos a utilizar o Mistral Large 2 para aplicar um padrão RAG sobre um documento de texto. A pergunta está escrita em coreano e questiona sobre as atividades do autor antes da universidade.

Utiliza o modelo de embeddings da Cohere para criar embeddings tanto do documento de texto como da pergunta. Para este exemplo, é utilizado o pacote faiss em Python como repositório de vetores.

O prompt enviado ao modelo Mistral inclui tanto as perguntas como os excertos recuperados que são semelhantes à pergunta. O modelo, então, fornece uma resposta em linguagem natural.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
O Mistral Small é outro modelo da família Mistral, inserido na categoria premier/empresarial. Como o nome indica, este modelo é um Small Language Model (SLM). As vantagens de utilizar o Mistral Small são:
- Poupança de custos em comparação com os LLMs da Mistral, como o Mistral Large e o NeMo – redução de preço de 80%
- Baixa latência – respostas mais rápidas em comparação com os LLMs da Mistral
- Flexível – pode ser implementado em diferentes ambientes com menos restrições ao nível dos recursos necessários.

O Mistral Small é ideal para:
- Tarefas baseadas em texto, como sumarização, análise de sentimento e tradução.
- Aplicações onde são feitos pedidos frequentes, devido à sua relação custo-benefício
- Tarefas de código de baixa latência, como revisão e sugestões de código


## Comparação entre Mistral Small e Mistral Large

Para mostrar as diferenças de latência entre o Mistral Small e o Large, executa as células abaixo.

Deves notar uma diferença nos tempos de resposta entre 3 a 5 segundos. Observa também os comprimentos e o estilo das respostas para o mesmo prompt.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Em comparação com os outros dois modelos discutidos nesta lição, o Mistral NeMo é o único modelo gratuito com licença Apache2.

É visto como uma evolução do anterior LLM open source da Mistral, o Mistral 7B.

Outras características do modelo NeMo incluem:

- *Tokenização mais eficiente:* Este modelo utiliza o tokenizador Tekken em vez do mais comum tiktoken. Isto permite um melhor desempenho em mais línguas e código.

- *Ajuste fino:* O modelo base está disponível para ajuste fino. Isto oferece mais flexibilidade para casos em que seja necessário personalizar o modelo.

- *Chamada de Funções Nativa* - Tal como o Mistral Large, este modelo foi treinado para chamadas de funções. Isto torna-o único por ser um dos primeiros modelos open source a oferecer esta funcionalidade.


## Mistral NeMo

Em comparação com os outros dois modelos discutidos nesta lição, o Mistral NeMo é o único modelo gratuito com licença Apache2.

É visto como uma evolução do anterior LLM open source da Mistral, o Mistral 7B.

Outras características do modelo NeMo incluem:

- *Tokenização mais eficiente:* Este modelo utiliza o tokenizador Tekken em vez do mais comum tiktoken. Isto permite um melhor desempenho em mais línguas e código.

- *Ajuste fino:* O modelo base está disponível para ajuste fino. Isto oferece mais flexibilidade para casos em que possa ser necessário personalizar o modelo.

- *Chamada de Funções Nativa* - Tal como o Mistral Large, este modelo foi treinado para chamada de funções. Isto torna-o único por ser um dos primeiros modelos open source a oferecer esta funcionalidade.


### Comparar Tokenizadores

Neste exemplo, vamos ver como o Mistral NeMo faz a tokenização em comparação com o Mistral Large.

Ambos os exemplos usam o mesmo prompt, mas deve reparar que o NeMo devolve menos tokens do que o Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## A aprendizagem não termina aqui, continua a tua jornada

Depois de terminares esta lição, consulta a nossa [coleção de aprendizagem sobre IA Generativa](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) para continuares a aprofundar os teus conhecimentos em IA Generativa!



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos pela precisão, tenha em atenção que as traduções automáticas podem conter erros ou imprecisões. O documento original, na sua língua nativa, deve ser considerado a fonte autorizada. Para informações críticas, recomenda-se a tradução profissional humana. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações incorretas resultantes da utilização desta tradução.
